<a href="https://colab.research.google.com/github/im-vne/Snakes-or-No-Snakes/blob/main/DataExploration_06_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### To gain authorization to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os # work with system    
import cv2 # work with images
import numpy as np # arrays and numerical analysis  
import matplotlib.pyplot as plt # for data plots
from PIL import Image # extract image information
import sklearn.linear_model # linear models
from sklearn.model_selection import train_test_split # data splitting
from sklearn import metrics # model evaluation
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import random # for random sampling 
import pathlib
import datetime
import time
import platform
import pandas as pd
import seaborn as sns


In [3]:
# check current wd
os.getcwd()
# set working directory in data folder
os.chdir("drive/Shareddrives/Capstone 2023/Data/MicrosoftSnakeAlgorithmProject")

In [16]:
# import the necessary packages
class LocalBinaryPatterns:
	def __init__(self, numPoints, radius):
		# store the number of points and radius
		self.numPoints = numPoints
		self.radius = radius
	def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
		lbp = feature.local_binary_pattern(image, self.numPoints,
			self.radius, method="uniform")
		(hist, _) = np.histogram(lbp.ravel(),
			bins=np.arange(0, self.numPoints + 3),
			range=(0, self.numPoints + 2))
		# normalize the histogram
		hist = hist.astype("float")
		hist /= (hist.sum() + eps)
		# return the histogram of Local Binary Patterns
		return hist, lbp

In [27]:
counter = 0 # if desired to break loop
# labels and filenames should match since they are appended in order

filenames = []
labels = []
# walk through all directories and files in current working directory
for root, dir, files in os.walk(os.getcwd()):
    
    for jpg in files:
        # if jpg file, then keep filename
        if ".JPG" in jpg:
            filenames.append(os.path.join(root, jpg))
            # for image files, create label based on path: 0 means empty, 1 means snake
            if "Empty" in root:
                labels.append(0)
            else:
                labels.append(1)

    counter += 1
    # if counter > 100: break

# save path for faulty image
faulty_image = '/content/drive/Shareddrives/Capstone 2023/Data/MicrosoftSnakeAlgorithmProject/Snake_Images/FL_Snakes/WF8/IMG_7087.JPG'


# check lengths of vectors
#print(len(labels))
#print(len(filenames))

# delete faulty index and its corresponding label
if faulty_image in filenames: 
  index = filenames.index(faulty_image)
  print(index)
  print(filenames[index])
  print(labels[index])
  del filenames[index]
  del labels[index]
  
# check and make sure only one as deleted
print(len(filenames))
print(len(labels))

# initialize image path 
images = [] 
counter2 = 0

# initialize timestamp 
timestamps = []
# gray scale (each is a 2d array)
#for img in filenames:
#    images.append(cv2.imread(img, cv2.IMREAD_GRAYSCALE))
#    counter2 += 1
    # if counter2 > 100: break

23021
/content/drive/Shareddrives/Capstone 2023/Data/MicrosoftSnakeAlgorithmProject/Snake_Images/FL_Snakes/WF8/IMG_7087.JPG
1
23275
23275


In [ ]:
#if you ran this once, make sure to make the timestamp vector empty or else it will keep adding past 23000 ish
timestamps = []

# takes timestamp from every image file (23275)
for img in filenames:
  image = Image.open(img)
  exif = image.getexif() # get meta data
  time = exif[306] # 306 is for the specific index for original image time
  timestamps.append(time) # save the times into a vector

#to check length and number of vectors
print(timestamps[1])
#print(timestamps[20])
print(labels[1])
#print(labels[20])
print(len(labels))
print(len(timestamps))


  

In [28]:
## plot times in which there were snakes cumulative ig?
#df = pd.DataFrame({'Snake Presence':labels, 'Date':timestamps})

#sns.lineplot(x = "Date", y = "Snake Presence", data = df)